In [70]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn


import utils as ut
import model_make as md
import run
import args

import time
from fastprogress import master_bar, progress_bar


import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim

import torchvision.transforms as transforms
import torchvision

from PIL import Image

from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
!pip install torchxrayvision

In [71]:
train_df, val_df, test_df = ut.dataloader()

In [72]:
train_df

,Path,LabelOne,LabelZero,LabelMulti
0,CheXpert-v1.0-small/train/patient00143/study2/...,"[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 0, 1]"
1,CheXpert-v1.0-small/train/patient00140/study13...,"[1, 0, 1, 0]","[1, 0, 1, 0]","[1, 0, 1, 0]"
2,CheXpert-v1.0-small/train/patient00294/study14...,"[0, 0, 0, 1]","[0, 0, 0, 1]","[0, 0, 0, 1]"
3,CheXpert-v1.0-small/train/patient00314/study15...,"[0, 1, 0, 1]","[0, 1, 0, 1]","[0, 1, 0, 1]"
4,CheXpert-v1.0-small/train/patient00314/study9/...,"[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
...,...,...,...,...
864,CheXpert-v1.0-small/train/patient00420/study2/...,"[0, 1, 0, 1]","[0, 1, 0, 1]","[0, 1, 0, 1]"
865,CheXpert-v1.0-small/train/patient00271/study2/...,"[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
866,CheXpert-v1.0-small/train/patient00310/study25...,"[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
867,CheXpert-v1.0-small/train/patient00210/study1/...,"[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]"


In [73]:
def main():
    train_df, val_df, test_df = ut.dataloader()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    img_path = r"E:\Courses"
    train_dataset = ut.ChestXrayDataset(img_path,train_df, args.IMAGE_SIZE, True,'u-ones')
    val_dataset = ut.ChestXrayDataset(img_path, val_df, args.IMAGE_SIZE, True,'u-ones')

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=args.BATCH_SIZE, shuffle=True, 
                                  num_workers=0, pin_memory=True)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=args.BATCH_SIZE, shuffle=False, 
                                num_workers=0, pin_memory=True)
    
    model = md.DenseNet121(num_classes=args.NUM_CLASSES).to(device)
    
    # Loss function
    loss_criteria = nn.BCELoss()

    # Adam optimizer
    optimizer = optim.Adam(model.parameters(), lr=args.LEARNING_RATE, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5)

    # Learning rate will be reduced automatically during training
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=args.LEARNING_RATE_SCHEDULE_FACTOR, 
                                                        patience=args.LEARNING_RATE_SCHEDULE_PATIENCE, 
                                                        mode='max', verbose=True)
    
    best_score = 0
    model_path = "densenet.pth"
    training_losses = []
    validation_losses = []
    validation_score = []
    validation_accuracy = []


    # Config progress bar
    mb = master_bar(range(5))
    mb.names = ['Training loss', 'Validation loss', 'Validation AUROC']
    x = []

    nonimproved_epoch = 0
    start_time = time.time()

    # Training each epoch
    for epoch in mb:
        mb.main_bar.comment = f'Best AUROC score: {best_score}'
        x.append(epoch)

        # Training
        train_loss = run.epoch_training(epoch, model, train_dataloader, device, loss_criteria, optimizer, mb)
        mb.write('Finish training epoch {} with loss {:.4f}'.format(epoch, train_loss))
        training_losses.append(train_loss)

        # Evaluating
        val_loss, new_score, new_acc = run.evaluating(epoch, model, val_dataloader, device, loss_criteria, mb)
        mb.write('Finish validation epoch {} with loss {:.4f}, AUROC score {:.4f} and accuracy {:.4f}'.format(epoch, 
                                                                                                              val_loss, 
                                                                                                              new_score,
                                                                                                              new_acc))
        validation_losses.append(val_loss)
        validation_score.append(new_score)
        validation_accuracy.append(new_acc)

        # Update learning rate
        lr_scheduler.step(new_score)

        # Update training chart
        mb.update_graph([[x, training_losses], [x, validation_losses], [x, validation_score]], [0,epoch+1], [0,1])

        # Save model
        if best_score < new_score:
            mb.write(f"Improve AUROC from {best_score} to {new_score}")
            best_score = new_score
            nonimproved_epoch = 0
            torch.save({"model": model.state_dict(), 
                        "optimizer": optimizer.state_dict(), 
                        "best_score": best_score, 
                        "epoch": epoch, 
                        "lr_scheduler": lr_scheduler.state_dict()}, model_path)
        else: 
            nonimproved_epoch += 1
        if nonimproved_epoch > 10:
            break
            print("Early stopping")
        if time.time() - start_time > 3600*8:
            break
            print("Out of time")

In [74]:
if __name__ =='__main__':
    main()

Setting XRayResizer engine to cv2 could increase performance.
Setting XRayResizer engine to cv2 could increase performance.


TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>

In [ ]:
[[0.31629297 0.15592107 0.29681253 0.54696655]
 [0.3526758  0.1471397  0.23511569 0.6586677 ]

In [ ]:
incept = torchvision.models.inception_v3(weights='Inception_V3_Weights.DEFAULT')

In [ ]:
incept

In [ ]:
model_output_features = incept.fc.in_features

In [ ]:
model_output_features

In [ ]:
incept.fc = nn.Sequential(nn.Linear(model_output_features, 4),
                                    nn.Sigmoid())

In [ ]:
incept

In [ ]:
# Get the input dimension of last layer
model_output_features = self.model.fc.in_features

        # Replace last layer with new layer that have num_classes nodes, after that apply Sigmoid to the output
self.model.fc = nn.Sequential(nn.Linear(model_output_features, num_classes),
                                              nn.Sigmoid())